# Final Project: Budget Planner App
### - CS110
### - Mohamed Gaber

## Introduction

The Budget Planner Application is an application that uses concepts from CS110 to build a budget plan for users. The application starts by allowing the users to input their budget for the month and the amount of money they want to save. Then, the application asks the users to enter data about all the items they might spend money on during the month. The data in general are:
- The name of the item
- The cost of the item
- A priority value from 1 to 10 that represents how much the item is important for them (prioritizing)

The application then sorts the data based on the priority value and provide a budget plan for the users to tell them which items to buy in order to best use their budget. 

## Technical Implementation

#### The application has three main sections:

#### 1- Prioritized Budget Section:

This section creates the budget plan by dividing the items that the user might needs into two sections: Needs and Wants, and then the needs will be divided into more subsections to shelter, food, education and other where each one comes before the other for example, the shelter is more important than food, then food is more important than education.. and all the needs sections are more important than the wants. The application allows the user to fill these fields. Each field will be stored in a separate binary tree data structure, (for example, when the user enters an item in the food field and clicks add, that item will be a node in the food tree that will get inserted depending on the priority value that the user assigns). So the application will have five different binary trees for (shelter, food, edcucation, other, and wants) (Note that wants have only one tree because it's not necessary to divide it into more sections)

Then to generate the budget plan, the application will do an inorder traversal for each tree (starting from the most important one which is the shelter and up to wants) so that the items get ordered from the highest priority value to the lowest priority value and are all added to one big list. Then starting from the first item in the list and according to the prices and the budget, the application will return the items that can be bought that fits the budget.

The binary tree data structure was chosen because it can both store data and return the data sorted. The tree will take an O(h) runtime to insert a node where h is the height of the tree which will be a logn in average case, and because we store n items in each tree, the average rumtime for storing data to the trees will be O(nlogn), and assuming that we enter an anverage of n items in each tree of the five trees, then we will have a run time of O(5*nlogn) which is O(nlogn). Then traversing the tree to sort the items takes a runtime of O(n) which for five trees will take a run time of O(5n). Then after generating the sorted list, the application uses a for loop to collect all the items in the list until the budget ends. So the final runtime of the prioritized budget section is O(n+n+nlogn) and the space complexity is O(n). This runtime is actually negligible because for the nature of the application as a budget planner, it will never deal with big data and the maximum number of items that any normal user might need for a month, shouldn't exceed a thousand. So knowing that the runtime doesn't matter, the binary tree structure is best here. Please note that the worst case runtime of binary search trees is O(n^2) in the case when all nodes form one height so it would be better to use red black trees or AVL trees to guarantee a balance, but hence the runtime doesn't matter and the binary search trees are easier to implement and offer the feature of sorting, the BST is the best data strucutre to use for this application. 

In [1]:
class Node: #defining a class node to make every item a node then insert it to a tree
    def __init__(self, name, val, cost): #setting the values of the node:
        self.l_child = None #left, right child and the parent are None by default until the node gets inserted or attached to another node
        self.r_child = None
        self.parent = None
        self.data = val #that's the priority value
        self.name = name
        self.cost = cost 

def insert(root, node): #function to insert a node to a tree
    if root is None:
        root = node
    else:
        if root.data > node.data: #following the binary tree structure by adding the smaller to the left
            if root.l_child is None: 
                root.l_child = node
                node.parent = root
            else:
                insert(root.l_child, node)
        else: #following the binary tree structure by adding the smaller to the left
            if root.r_child is None:
                root.r_child = node
                node.parent = root
            else:
                insert(root.r_child,node)#using recursion in case the tree has more than three other nodes
    return root



def inorder(root, lis): #The sorting algorithm that I will use (traversing the tree)
    if root != None:
        inorder(root.l_child, lis) #going from the most node in the left until the most node in the right
        lis.append(root) #adding the nodes to the list in a sorted order according to their priority values
        inorder(root.r_child, lis)
    return lis

#### 2- Greedy Budget Section:

Unlike the prioritized budget section, the greedy budget section is a litte bit more flexible because it doesn't divide into two sections of needs and wants and it allows the user to enter an item and then store it in one tree. However, this section follows a different algorithm, beside the priority value for each item, the application will ask the user to enter the cost per unit for each item. For example, if that item is meat then the user will enter 10 dollars per kilogram, if it's visiting cinmena, the user should enter 12 dollars per ticket. The user then should enter the needed number of units for each item for the month. The application will then follow the same method as above to store the data in one tree and then sort it in terms of priorty values and then the new thing here is that the application will use the below greedy algorithm code to get as much units as possible that fits the budget. 

Greedy algorithm was appropriate here for two reasons:
- The problem has an optimal substructure because the optimal solution (which is getting maximum number of items with highest value of priority) will be the sum of the optimal solutions for subproblems (which are finding highest priority value items that fit the budget.

- The problem has the greedy choice property because by sorting the items in ascending order in terms of the highest priority value, and then choosing to start with the one that has highest value and moving forward, the global optimal solution will be arrived at by selecting the local optimal solution at each step.

So by using this greedy algorithm, the application will be able to tell the user how many units of each item to buy in order to achieve highest priority and fit the budget to the fullest. The difference between this section and the prioritized section is that the prioritized section makes it more prioritized by constructing five trees for five fields where each field has more priority than the previous one, while the greedy section makes it more flexible and return more detailed budget plan (number of units)

That said, the run time of the application for the greedy section will have the same run time for inserting items and sorting them which is O(n+nlogn) plus the time for the greedy algorithm below which takes O(n) time. So the total run time is then O(n+n+nlogn) and a space complexity of O(n) which is again negligible because of the small number of items.

In [2]:
class Node2: #defining another class for nodes to include the number of items and the cost per item in this one
    def __init__(self, name, val, cost_per_item, num_items): 
        self.l_child = None
        self.r_child = None
        self.parent = None
        self.data = val #the priority value
        self.name = name
        self.cost = cost_per_item #cost per item for the greedy algorithm
        self.num = num_items #number of items
        
def greed(budget, Nodes):#the greedy algorithm that takes a sorted list of nodes and the budget
    i=0 #will use this index to iterate
    lis = []
    while budget > 0 and i < len(Nodes): #iterating over the nodes to fill my budget
        if budget >= (Nodes[i].num)*(Nodes[i].cost): #if budget is more than the cost of all my needs of this item
            lis.append((str(Nodes[i].name)+': '+str(Nodes[i].num)+" Units"))#append the name and number of units
            budget = budget -(Nodes[i].num)*(Nodes[i].cost)#update budget
            i+=1#iterate
        elif budget//Nodes[i].cost >0: #check to see if I can buy any units of this item instead of all of it
            lis.append((str(Nodes[i].name)+': '+str(budget//Nodes[i].cost)+" Units"))
            budget = budget -(budget//Nodes[i].cost)*(Nodes[i].cost)
            i+=1
        else:
            i+=1
            
    return lis , budget #returning the list of items and the remaining of the budget if any

#### 3- Maximum Values Section:

This section is customized for the wants of the users (the stuff that are not necessities of life). The user will enter his wants with values from 1 to 10 on how much he prefers each one, and then the application will return a list of the items that achieve a maximum utility. For example, if your desire for travelling to a different country is 8 but travelling is expensive and will consume all of your budget. At the same time you have a desire of 7 of going to Cinema and a desire of 7 of going to a theme park and a desire of 2 of buying desserts, then the application will suggest doing the 7, 7, 2 which together will have a value of 16 so doing them together will be better than just traveling. The code below shows how the application does that using dynamic programming. Please note that if you added the same items to the greedy or the prioritized section, they would choose traveling at first place because they consider the value 8 as the first priority that must be achieved unlike this maximum values section that tries to achive the highest utility.

The reason I used dynamic programming here is:

- The problem has an optimal substrucure because the global optimal solution(finding items that achieve highest value while satisfying the budget) is the sum of the optimal solutions to the subproblems and the global optimal solution will contain within it optimal solutions to the subproblems.
- The problem has overlapping subproblems because the same subproblem will occur more often and that's why I used a table to store the subproblems and use it to solve other subproblems. (please note that because the application nature doesn't allow many number of items, there will be small number of overlapping subproblems).

This problem is just like the Knapsack problem where the weight, the knapsack can carry, is the budget and the weights of the items is the cost here. And so while the knapsack algorithm looks for the highest value items that fit the knapsack, this algorithm works to find the highest priority values that fit the budget.


The dynamic programming solution here will take a run time of O(n^2) to find the highest values and then O(n) to find the names of the items then a total runtime of O(n^2+n) = O(n^2) adding that to the time that the tree takes to insert values, the total runtime of the application in that section is then O(n^2 + nlogn) (note that there is no traversing here) the runtime is then O(n^2) with a space complexity of O(budget) (because the budget will be used to make a table to store the solutions to the subproblems as shown in the code below.


In [3]:
def max_value(W, Nodes): #The dynamic programming algorithm that I used for the maximum values section
    n = len(Nodes)
    K = [[0 for w in range(W + 1)] 
            for i in range(n + 1)] #bulding a table to hold the solutions to the subprobelm in a
    #bottom up manner
    items = [] #this list will contain the items
              
    for i in range(n + 1): #using two for loops to solve the subproblems and get values of the items
        for w in range(W + 1): 
            if i == 0 or w == 0: 
                K[i][w] = 0
            elif Nodes[i - 1].cost <= w: #making sure that the cost of each item is less than my budget
                K[i][w] = max(Nodes[i - 1].data  #comparing different subproblems to get the optimal subsolution
                  + K[i - 1][w - Nodes[i - 1].cost], 
                               K[i - 1][w]) 
            else: 
                K[i][w] = K[i - 1][w] 
  
     
    last = K[n][W] #Using the result I got from above, I will get the items names (using their values)
    w = W #setting w to my budget
    for i in range(n, 0, -1): 
        if last <= 0: #if no item was included
            break 
        if last == K[i - 1][w]: 
            continue
        else: 
  
             
            items.append(Nodes[i - 1].name) #in this case, the item was included and then I add it to list
              
            
            last = last - Nodes[i - 1].data #deducing its values and weight to find the names of the other items
            w = w - Nodes[i - 1].cost
    return items #return the final list that has the names of the items
class Node:
    def __init__(self, name, val, cost):
        self.l_child = None
        self.r_child = None
        self.parent = None
        self.data = val
        self.name = name
        self.cost = cost
        
Nodes = [Node("travelling", 8, 100),Node("Cinema", 7, 40),Node("Theme_park", 7, 50),Node("desserts", 2, 10) ] 
budget = 100
max_value(budget, Nodes)

['desserts', 'Theme_park', 'Cinema']

### References:

Geeksforgeeks Website: https://www.geeksforgeeks.org/